# Testing Pandas library

<h5>Using 2 scritps, 1 for Metric Display web page, 1 to create home made report based of Progress and ATA, ODA, etc box spreadsheet</h5>
<br>

### Script to delete the existent dtTasks_ file on downloads

In [45]:
import subprocess
subprocess.run("del /f C:\\Users\\016434613\\Downloads\\dtTasks_*.xlsx", shell=True, capture_output=True)

CompletedProcess(args='del /f C:\\Users\\016434613\\Downloads\\dtTasks_*.xlsx', returncode=0, stdout=b'', stderr=b'')

### Script to run with selenium master file link

In [1]:
from selenium import webdriver
import time

mydriver = webdriver.Chrome("chromedriver.exe")
mydriver.get("https://progress.us1a.cirrus.ibm.com/api/rpt/dt/1600277460000/ms/jas8n38daai/1/1600431650693")
time.sleep(20)
mydriver.close()

C:\Users\016434~1\AppData\Local\Temp/ipykernel_24292/1085608280.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  mydriver = webdriver.Chrome("chromedriver.exe")


In [2]:
import pandas as pd
import numpy as np

import glob
import re

filepath = r"C:\Users\016434613\Downloads"
filepath_ = ""


# Acá estamos obteniendo todos los files que terminen con .xlsx
all_files = glob.glob(filepath + "\\*.xls*")
for f in all_files:
    # de esa lista buscamos especificamente el que tenga \\dtTasks_ para agarrar nuestro file
    s = re.search(r"\\dtTasks_",f)
    if s != None:
        filepath_ = f

#Usamos la ruta que obtuvimos arriba
if filepath_ != "":
    df = pd.read_excel(filepath_,sheet_name="DT Tasks")
df = df.rename(columns={'ROBOT NUMBER':'ROBOT_NUMBER'})

df_lindo = df.loc[:,['ORIGIN RELEASE','STATUS','ROBOT_NUMBER','TITLE','AUTOMATION TYPE','REGIONS','MANUAL EXECUTION DURATION (hours per year)']]
df_lindo = df_lindo.rename(columns={'MANUAL EXECUTION DURATION (hours per year)':'SAVING HS'})

df_ATA = pd.read_excel(r"C:\Users\016434613\Box\ATA Other\ATA-ODA-WBA-RPA number assigment.xlsx",sheet_name="ATA")
df_ATA = df_ATA.rename(columns={'Task #':'ROBOT_NUMBER'})


#########################################################################
#########################################################################

df3 = df_lindo
df_ATA_2 = df_ATA.loc[:,['ROBOT_NUMBER','Developer']]


df4 = df3.merge(df_ATA_2, on="ROBOT_NUMBER", how="left")
df4['ORIGIN RELEASE'] = df4['ORIGIN RELEASE'].apply(lambda x: str(x))
df4['QUARTER'] = df4['ORIGIN RELEASE'].apply(lambda x: x[4:7]+x[11:15])

#df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '4Q' if x == 'Dec2021' else '4Q' if x == 'Nov2021' else '4Q' if x == 'Oct2021' else 'Other')
df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '1Q' if x == 'Jan2022' else '1Q' if x == 'Feb2022' else '1Q' if x == 'Mar2022' else 'Other')


df5 = df4.loc[df4['QUARTER']=='1Q',]
df5.drop(columns=["ORIGIN RELEASE"],inplace=True)


#########################################################################
#########################################################################

df_ODA = pd.read_excel(r"C:\Users\016434613\Box\ATA Other\ATA-ODA-WBA-RPA number assigment.xlsx",sheet_name="ODA")
df_ODA = df_ODA.rename(columns={'Task #':'ROBOT_NUMBER'})
df_ODA = df_ODA.loc[:,['ROBOT_NUMBER','Developer']]
df5 = df5.merge(df_ODA, on="ROBOT_NUMBER", how="left")

df5['Developer'] = df5.apply(lambda x: x['Developer_x'] if str(x['Developer_y'])=="nan" else x['Developer_y'],axis=1)
df5.drop(columns=['Developer_y','Developer_x'],inplace=True)

#df5.loc[df5['ROBOT_NUMBER']=="ATA000",'Developer']="Michele Lobina"

C:\Users\016434613\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Script to obtain mgr reports from bluepages API 

In [3]:
from urllib import request as req
import json

mgrserials = ['016434613','A01130693','A51733693','900688897']
mgrname = {'016434613':'Martin Campos','A01130693':'Marek Tarkos','A51733693':'Andrej Csiaki','900688897':'Melissa Bledsoe'}
devnames = {}
for mgr in mgrserials:
    request = req.urlopen("https://unified-profile-api.us-south-k8s.intranet.ibm.com/v3/profiles/" + mgr + "/teamResolved")
    datos = request.read().decode()
    dictdatos = json.loads(datos)
    devlist = dictdatos['content']['functional']['reports']
    for dev in devlist:
        #method setdefault(key,[]) sirve para poner una primera key con una lista vacia, si no, tira error de que no existe la key
        devnames.setdefault(mgrname[mgr], []).append(dev['nameDisplay'])

In [4]:
### Create Lists ###
devlistmc = devnames['Martin Campos']
devlistmt = devnames['Marek Tarkos']
devlistac = devnames['Andrej Csiaki']
devlistmb = devnames['Melissa Bledsoe']

### Create 2 dimension df, 1 column with the list, second column value fixed de Mgr ###
df6 = pd.DataFrame(devlistmc, columns=['Developer'])
df6.loc[df6['Developer']=="Pablo (Paul) Iacovone",'Developer']="Pablo Iacovone"
df6.loc[df6['Developer']=="Santiago Akira Kitashima",'Developer']="Santiago Kitashima"
df6.loc[df6['Developer']=="Ezequiel Ricardo Ferlauto",'Developer']="Ezequiel Ferlauto"
df6.loc[df6['Developer']=="Andrés Lopez Grosman",'Developer']="Andres Grosman"
df6.loc[df6['Developer']=="Dario Cesar Atach",'Developer']="Dario Atach"
df6['Manager'] = "Martin Campos"

df7 = pd.DataFrame(devlistmt, columns=['Developer'])
df7.loc[df7['Developer']=="Viera Babčancová",'Developer']="Viera Babcancova"
df7.loc[df7['Developer']=="Milan Babčanec",'Developer']="Milan Babcanec"
df7.loc[df7['Developer']=="Adam Balák",'Developer']="Adam Balak"
df7.loc[df7['Developer']=="Peter Blahut",'Developer']="Peter Blahut"
df7.loc[df7['Developer']=="Viera Csiaki",'Developer']="Viera Csiaki"
df7.loc[df7['Developer']=="Barbora Depešová",'Developer']="Barbora Depesova"
df7.loc[df7['Developer']=="Richard Hantke",'Developer']="Richard Hantke"
df7.loc[df7['Developer']=="Lenka Holáňová",'Developer']="Lenka Holanova"
df7.loc[df7['Developer']=="Andrea Hrčková",'Developer']="Andrea Hrckova"
df7.loc[df7['Developer']=="Miroslav Kocian",'Developer']="Miroslav Kocian"
df7.loc[df7['Developer']=="Kristína Margetinová",'Developer']="Kristina Margetinova"
df7.loc[df7['Developer']=="Lucia Mikuš",'Developer']="Lucia Mikus"
df7.loc[df7['Developer']=="Soňa Olšová",'Developer']="Sona Olsova"
df7.loc[df7['Developer']=="Marta Pancotti",'Developer']="Marta Pancotti"
df7.loc[df7['Developer']=="Nina Tadlánková",'Developer']="Nina Tadlankova"
df7.loc[df7['Developer']=="Ivan Trubač",'Developer']="Ivan Trubac"
df7.loc[df7['Developer']=="Kristína Vojtová",'Developer']="Kristina Vojtova"
df7.loc[df7['Developer']=="Lubomir Cmiko (Lubo)",'Developer']="Lubomir Cmiko"
df7.loc[df7['Developer']=="Radomíra Šulová",'Developer']="Radomira Sulova"
df7['Manager'] = "Marek Tarkos"

df8 = pd.DataFrame(devlistac, columns=['Developer'])
df8.loc[df8['Developer']=="Marián Chrást",'Developer']="Marian Chrast"
df8.loc[df8['Developer']=="Massimo Cireddu",'Developer']="Massimo Cireddu"
df8.loc[df8['Developer']=="Katarina Kadlecova",'Developer']="Katarina Kadlecova"
df8.loc[df8['Developer']=="Marta Kapolková",'Developer']="Marta Kapolkova"
df8.loc[df8['Developer']=="Nikolaj Kniha",'Developer']="Nikolaj Kniha"
df8.loc[df8['Developer']=="Veronika Krchnáková",'Developer']="Veronika Krchnakova"
df8.loc[df8['Developer']=="Michele Lobina",'Developer']="Michele Lobina"
df8.loc[df8['Developer']=="Vladimír Maťo",'Developer']="Vladimir Maťo"
df8.loc[df8['Developer']=="Veronika Miklovičová",'Developer']="Veronika Miklovicova"
df8.loc[df8['Developer']=="Richard Nerodolík",'Developer']="Richard Nerodolik"
df8.loc[df8['Developer']=="Micheál O'Reilly",'Developer']="Mícheál O'Reilly"
df8.loc[df8['Developer']=="Michala Ožóg",'Developer']="Michala Ozog"
df8.loc[df8['Developer']=="Branislav Podoba",'Developer']="Branislav Podoba"
df8.loc[df8['Developer']=="Andre Daniel Rodrigues",'Developer']="Andre Rodrigues"
df8.loc[df8['Developer']=="Terézia Straková",'Developer']="Terezia Strakova"
df8.loc[df8['Developer']=="Lenka Tarkošová",'Developer']="Lenka Tarkosova"
df8.loc[df8['Developer']=="Valentína Vargová",'Developer']="Valentina Vargova"
df8.loc[df8['Developer']=="Marek Štípala",'Developer']="Marek Stipala"
df8['Manager'] = "Andrej Csiaki"

df9 = pd.DataFrame(devlistmb, columns=['Developer'])
df9.loc[df9['Developer']=="Gabe Hutson",'Developer']="Gabriel Hutson"
df9['Manager'] = "Melissa Bledsoe"


### Append results into one df - df10 ####
df10 = df6
df10 = df10.append(df7)
df10 = df10.append(df8)
df10 = df10.append(df9)

In [5]:
df5 = df5.merge(df10, on="Developer", how="left")
#df5.loc[df5["ROBOT_NUMBER"]=="ODA112","Developer"]="Bruno Secchiari"
#df5.loc[df5["ROBOT_NUMBER"]=="ODA127","Developer"]="Bruno Secchiari"
df5['Count']=1
df5 = df5.loc[:,['Manager','Developer','Count','SAVING HS','QUARTER','ROBOT_NUMBER','TITLE','STATUS','AUTOMATION TYPE','REGIONS']]
df5.to_excel(r"C:\Users\016434613\Desktop\Home Made Report.xlsx", index=False)

#### Para buscar en progress con el task id: https://progress.us1a.cirrus.ibm.com/digital-transformation/task/

### Backlog Report

In [6]:
df_Back = pd.read_excel(filepath_,sheet_name="DT Tasks")
df_Back = df_Back.rename(columns={'DSL/SUPERVISOR (NAME)':'DSL','ROBOT NUMBER':'ROBOT_NUMBER','AUTOMATION TYPE':'AUTOMATION_TYPE','MANUAL EXECUTION DURATION (hours per year)':'SAVING HS'})
df_Back = df_Back.loc[:,['TASK ID','CERTIFIED','DSL','STATUS','ROBOT_NUMBER','TITLE','AUTOMATION_TYPE','REGIONS','SAVING HS']]
df_Back = df_Back.loc[df_Back['STATUS']=="Ready for development"]
df_Back['CERTIFIED'] = df_Back['CERTIFIED'].apply(lambda x: str(x))
df_Back['CERTIFIED'] = df_Back['CERTIFIED'].apply(lambda x: x[4:7]+" "+x[11:15])
df_Back = df_Back.loc[df_Back['ROBOT_NUMBER'].isnull()==True]
df_Back.to_excel(r"C:\Users\016434613\Desktop\Backlog.xlsx", index=False)

### Specification Report

In [21]:
df_Spec  = pd.read_excel(filepath_,sheet_name="DT Tasks")
df_Spec = df_Spec.rename(columns={'DSL/SUPERVISOR (NAME)':'DSL','DSL/SUPERVISOR (EMAIL)':'DSL EMAIL','ROBOT NUMBER':'ROBOT_NUMBER','AUTOMATION TYPE':'AUTOMATION_TYPE','MANUAL EXECUTION DURATION (hours per year)':'SAVING HS'})
df_Spec = df_Spec.loc[:,['TASK ID','CERTIFIED','DSL','DSL EMAIL','STATUS','ROBOT_NUMBER','TITLE','AUTOMATION_TYPE','REGIONS','SAVING HS']]
df_Spec = df_Spec.loc[df_Spec['STATUS']=="Specification"]
df_Spec['CERTIFIED'] = df_Spec['CERTIFIED'].apply(lambda x: str(x))
df_Spec['CERTIFIED'] = df_Spec['CERTIFIED'].apply(lambda x: x[4:7]+" "+x[11:15])
df_Spec.loc[df_Spec['CERTIFIED']=='fin ','CERTIFIED']=""
df_Spec = df_Spec.loc[df_Spec['ROBOT_NUMBER'].isnull()==True]
df_Spec.drop(columns=['ROBOT_NUMBER','STATUS'],inplace=True)
df_Spec.to_excel(r"C:\Users\016434613\Desktop\Specification.xlsx", index=False)

### Validation Report

In [20]:
df_Val = pd.read_excel(filepath_,sheet_name="DT Tasks")
df_Val = df_Val.rename(columns={'DSL/SUPERVISOR (NAME)':'DSL','DSL/SUPERVISOR (EMAIL)':'DSL EMAIL','ROBOT NUMBER':'ROBOT_NUMBER','AUTOMATION TYPE':'AUTOMATION_TYPE','MANUAL EXECUTION DURATION (hours per year)':'SAVING HS'})
df_Val = df_Val.loc[:,['TASK ID','CERTIFIED','DSL','DSL EMAIL','STATUS','ROBOT_NUMBER','TITLE','AUTOMATION_TYPE','REGIONS','SAVING HS']]
df_Val = df_Val.loc[df_Val['STATUS']=="Validation"]
df_Val['CERTIFIED'] = df_Val['CERTIFIED'].apply(lambda x: str(x))
df_Val['CERTIFIED'] = df_Val['CERTIFIED'].apply(lambda x: x[4:7]+" "+x[11:15])
df_Val.loc[df_Val['CERTIFIED']=='fin ','CERTIFIED']=""
df_Val = df_Val.loc[df_Val['ROBOT_NUMBER'].isnull()==True]
df_Val.drop(columns=['ROBOT_NUMBER','STATUS'],inplace=True)
df_Val.to_excel(r"C:\Users\016434613\Desktop\Validation.xlsx", index=False)